# Greenland, 2004-2023, Max: 205, Min: 305

Latitude: 59.79 to 83.63, approximately 25 degrees
Longitude: -73.00 to -12.00, approximately 60 degrees

In [4]:
import sys
sys.path.append('proposed/')
from proposed.query_executor_find_time2 import FindTimeExecutor

fqe = FindTimeExecutor(
    variable="2m_temperature",
    start_datetime="2010-01-01 00:00:00",
    end_datetime="2015-12-31 23:00:00",
    temporal_resolution="hour",
    max_lat = 85,
    min_lat = 60,
    min_lon = -75,
    max_lon = -10,
    spatial_resolution=0.25,
    aggregation="min",
    time_series_aggregation_method="min",
    filter_predicate=">",
    filter_value=240,
)
ft = fqe.execute()
ft

******************
whole year
2010
2011
2012
2013
2014
2015
whole month
whole day
whole hour
checking years
local files: ['/data/era5/agg/2m_temperature/2m_temperature-year-min_100.nc']
local files: ['/data/era5/agg/2m_temperature/2m_temperature-year-max_100.nc']
year: 2010, min: 219.9663310604587, max: 298.7164687885829
year: 2011, min: 213.21269194688182, max: 297.62591641690045
year: 2012, min: 209.38149398895652, max: 298.0616247675139
year: 2013, min: 209.50892114204288, max: 296.3175098699112
year: 2014, min: 217.8889331305377, max: 300.0018084228926
year: 2015, min: 210.8267861726465, max: 297.6451388441334
checking months
local files: ['/data/era5/agg/2m_temperature/2m_temperature-month-min_100.nc']
local files: ['/data/era5/agg/2m_temperature/2m_temperature-month-max_100.nc']
2010-05: min > filter, True
2010-06: min > filter, True
2010-07: min > filter, True
2010-08: min > filter, True
2010-09: min > filter, True
2011-07: min > filter, True
2011-08: min > filter, True
2012-06:

<xarray.Dataset> Size: 473kB
Dimensions:  (time: 52584)
Coordinates:
  * time     (time) datetime64[ns] 421kB 2010-01-01 ... 2015-12-31T23:00:00
Data variables:
    t2m      (time) bool 53kB False False False False ... False False False

In [ ]:
from pandas import Timestamp
range_1 = [[Timestamp('2010-01-01 00:00:00'), Timestamp('2010-12-31 23:00:00')]]
range_2 = [[Timestamp('2010-01-01 00:00:00'), Timestamp('2010-12-31 23:00:00')]]
range_3 = [[Timestamp('2010-01-01 00:00:00'), Timestamp('2010-04-30 23:00:00')], [Timestamp('2010-10-01 00:00:00'), Timestamp('2010-12-31 23:00:00')]]
hour_range = [[Timestamp('2010-01-01 00:00:00'), Timestamp('2010-04-27 23:00:00')], [Timestamp('2010-04-29 00:00:00'), Timestamp('2010-04-29 23:00:00')], [Timestamp('2010-10-04 00:00:00'), Timestamp('2010-10-12 23:00:00')], [Timestamp('2010-10-17 00:00:00'), Timestamp('2010-11-27 23:00:00')], [Timestamp('2010-12-02 00:00:00'), Timestamp('2010-12-31 23:00:00')]]

In [ ]:
def get_start_end(range):
    start = range[0][0]
    end = range[-1][1]
    return start, end

start, end = get_start_end(range_1)

from proposed.query_executor_get_raster import GetRasterExecutor
qe = GetRasterExecutor(
    variable="2m_temperature",
    start_datetime=start,
    end_datetime=end,
    temporal_resolution="hour",
    max_lat=85,
    min_lat=60,
    min_lon=-75,
    max_lon=-10,
    spatial_resolution=0.25,
    aggregation="min",
)
qe.execute_lazy()

In [ ]:
r = range_1[0]
r_start, r_end = r[0], r[1]
start = r_start.strftime("%Y-%m-%d %H:%M:%S")
end = r_end.strftime("%Y-%m-%d %H:%M:%S")
qe.execute_lazy().sel(time=slice(start, end))

In [ ]:
import xarray as xr

def func(range, temporal_res):
    total_start = range[0][0]
    total_end = range[-1][1]

    min_qe = GetRasterExecutor(
        variable="2m_temperature",
        start_datetime=total_start,
        end_datetime=total_end,
        temporal_resolution=temporal_res,
        max_lat=85,
        min_lat=60,
        min_lon=-75,
        max_lon=-10,
        spatial_resolution=0.25,
        aggregation="min",
    )
    ds_lazy = min_qe.execute_lazy()
    ds_list = []
    for r in range:
        r_start, r_end = r[0], r[1]
        start = r_start.strftime("%Y-%m-%d %H:%M:%S")
        end = r_end.strftime("%Y-%m-%d %H:%M:%S")
        ds = ds_lazy.sel(time=slice(start, end))
        ds_list.append(ds)
    return xr.concat(ds_list, dim="time")

In [ ]:
func(range_1, "year")

In [ ]:
func(range_2, "month")

In [ ]:
func(range_3, "day")

In [ ]:
func(hour_range, "hour")